# Prod - Flask with Llama.cpp

In [1]:
!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python -q
!pip install langchain -q
!pip install flask -q
!pip install flask_socketio -q

## Imports

In [2]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

## Building model

In [3]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

orca_path = "/Users/niccolominetti/Desktop/GTL-machine/llama.cpp/orca-2-7b.Q4_K_M.gguf"
code_llama_path = "/Users/niccolominetti/Desktop/GTL-machine/llama.cpp/codellama-7b-instruct.Q2_K.gguf"
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

In [4]:

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=orca_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx= 1024, # ctx_size=1024,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
    seed=81637,
    #repeat_penalty=1,
    
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/niccolominetti/Desktop/GTL-machine/llama.cpp/orca-2-7b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32003,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.

In [16]:
# Make sure the model path is correct for your system!
code_llm = LlamaCpp(
    model_path=code_llama_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx= 4096, # ctx_size=1024,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
    seed=81637,
    #repeat_penalty=1,
    
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/niccolominetti/Desktop/GTL-machine/llama.cpp/codellama-7b-instruct.Q2_K.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q2_K     [  4096, 32016,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q3_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q3_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q3_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q2_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output

## Agents
- Build a loop which keeps a system message
- Has an internal sliding window of infinite iterations
- Has an evaluation prompt which adds a score to each answer
- And only keeps the best answer in the system

Example

[Task] Evaluate the impact of introducing a new public transport system in a city.[\Task]
[Instructions] To assess the impact of a new public transport system:
1. Conduct a feasibility study focusing on demographic and geographic factors.
2. Analyze current traffic data to understand potential alleviation points.
3. Survey public opinion to gauge acceptance and expected usage.
4. Calculate environmental benefits, considering emission reductions and energy efficiency.
5. Develop a cost-benefit analysis including construction, operation, and maintenance costs.

In [5]:
def create_agent_prompt(task, best_answer="\n"):
    """
    Generates a prompt for a language model to provide instructions for accomplishing a specific task.

    Parameters:
    task (str): The task for which the instructions are to be generated.
    best_answer (str, optional): A pre-existing approach or solution to the task. Defaults to a newline.

    Returns:
    tuple: Contains the generated prompt, a marker indicating the end of the instructions section, 
           and the best_answer parameter value.

    The function creates a prompt template that frames the language model as an entity dedicated to expanding human knowledge.
    It instructs the model to outline a set of instructions for the specified task, and to enhance or detail an existing approach
    if provided in the best_answer section.
    """

    # Prompt build
    prompt = """[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
For the task specified below, outline a set of instructions to successfully achieve it.
If there's an existing approach in the [Best_answer] section, enhance or detail it for a more effective strategy.
[Best answer]\n""" + best_answer + """[\Best answer]
<</SYS>>

Examples

[Task] Solve a logical puzzle involving the arrangement of different colored balls in a specific order.[\Task]
[Instructions] To solve a logical puzzle involving colored balls:
1. Identify and list all given constraints and rules.
2. Start by placing balls that have the most constraints.
3. Use a process of elimination for balls with fewer constraints.
4. Check each step against the puzzle's rules to ensure compliance.
5. Iterate through different arrangements, using logical deductions until the correct order is achieved.[\Instructions]

[Task] """ + task + """[\Task][\INST]
[Instructions] """
    return prompt, "[\Instructions]"

In [6]:
def create_eval_prompt(task, response, best_answer):
    """
    Generates a prompt for evaluating two different options in response to a given task.

    Parameters:
    task (str): The task for which the options are to be evaluated.
    response (str): The first option (Option A) to be evaluated against the task.
    best_answer (str): The second option (Option B) considered as the existing best answer.

    Returns:
    tuple: Contains the generated evaluation prompt and a marker indicating the end of the evaluation score section.

    This function creates a prompt template where the language model acts as a cognitive evaluator.
    It presents two options (A and B) for a specified task and asks the model to select which option better answers the question or task.
    The function includes examples to guide the model in how to evaluate and select the more appropriate option for different tasks.

    Extra example
    [Task]: Describe the process to set up an effective remote work environment.[\Task]
    [Options]
    Option A
    Get a good computer and a reliable internet connection.
    Option B
    1. Invest in quality technology and a reliable internet connection.
    2. Set up a dedicated, ergonomically sound workspace.
    3. Establish clear remote work policies and communication guidelines.
    4. Implement tools for project management and team collaboration.
    5. Schedule regular check-ins and provide support for mental well-being.[\Options]
    [Answer] Option B[\Answer]
    """
    # Build eval prompt
    eval_prompt = f"""[INST] <<SYS>>You are a cognitive evaluator, select which option better answers the question or task.
<<\SYS>>
Examples:
[Task]: Outline the steps to implement a successful urban gardening project.[\Task]
[Options]
Option A
1. Select an area with good sunlight.
2. Prepare the soil with fertilizers.
3. Plant seeds or seedlings of various vegetables and herbs.
4. Set up a watering schedule.
5. Regularly check for pests and diseases.
Option B
1. Identify community needs and preferences for the garden.
2. Secure a location with proper sunlight and water access.
3. Collaborate with local gardening experts for plant selection and garden design.
4. Organize volunteer days for planting and maintenance.
5. Establish a system for sharing the harvest among contributors.[\Options]
[Answer] Option B[\Answer]

[Task]:{task}""" + r"[\Task]" + f"""
[Options]
Option A
{response}""" + f"\nOption B\n{best_answer}" + """[\INST]
[Answer]: Option"""
    eval_stop = r"\Evaluation score"

    return eval_prompt, eval_stop

## Orchestrate n results

In [7]:
import pandas as pd
import os
from datetime import datetime

def update_csv(filename, data):
    # Check if the file exists
    if os.path.exists(filename):
        # If the file exists, read it and append the new data
        df = pd.read_csv(filename)
        df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

        #df = df.append(data, ignore_index=True)
    else:
        # If the file doesn't exist, create a new DataFrame
        df = pd.DataFrame([data])

    # Save the updated DataFrame to the CSV file
    df.to_csv(filename, index=False)

In [8]:
### Loop function ###
def n_search(task, n=3, debug=False):
    
    best_answer = "\n"

    # Ensure the 'sessions' directory exists
    os.makedirs('flask/sessions', exist_ok=True)

    # Generate a unique filename based on the current date and time
    filename = f'flask/sessions/session_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'

    # First iteration
    # Build prompt and call model
    prompt, stop_words = create_agent_prompt(task=task, best_answer=best_answer)
    best_answer = llm(prompt, stop=[stop_words, "Thought", "[Output]","[\Task]", "[\Previous thought]","\INST", "[\Best answer]", "\n\n"])

    for i in range(n):
        
        # Build prompt and call model
        prompt, stop_words = create_agent_prompt(task=task, best_answer=best_answer)
        response = llm(prompt, stop=[stop_words, "Thought", "[Output]","[\Task]", "[\Previous thought]","\INST", "[\Best answer]", "\n\n"])
        # Build evaluation prompt and call model
        eval_prompt, eval_stop = create_eval_prompt(task=task, response=response, best_answer=best_answer)
        eval_response = llm(eval_prompt, stop=[eval_stop, "[\Answer]", "\n", "\n\n"])
        # Keep best result with fuzzy logic
        if debug: print(f"\n\nChallenge:\n{response}\nBest Answer so far:\n{best_answer}\nEvaluation:{eval_response}")

         # Data to be written to CSV
        data = {
            'iteration': i,
            'task': task,
            'best_answer': best_answer,
            'response': response,
            'eval_response': eval_response
        }

        # Update CSV file with the new data
        update_csv(filename, data)
        
        # Update best answer
        if "A" in eval_response:
            # The new answer is now our best answer
            best_answer = response

            
    return best_answer
    

In [9]:
task = "How do I write code that solve linear equations?"

In [10]:
result = n_search(task, n=3)

1. Identify the coefficients and constants of the equation.
2. Use a suitable programming language or software to create a function that takes the coefficients and constants as input.
3. Implement an algorithm to find the solution for x, using methods such as substitution, elimination, or graphical representation.
4. Test the code by entering different linear equations and verify if it produces the correct solutions.


llama_print_timings:        load time =    5034.68 ms
llama_print_timings:      sample time =      51.56 ms /    88 runs   (    0.59 ms per token,  1706.78 tokens per second)
llama_print_timings: prompt eval time =    5033.64 ms /   250 tokens (   20.13 ms per token,    49.67 tokens per second)
llama_print_timings:        eval time =    7655.78 ms /    87 runs   (   88.00 ms per token,    11.36 tokens per second)
llama_print_timings:       total time =   14043.44 ms
Llama.generate: prefix-match hit


1. Identify the coefficients and constants of the equation. 2. Use a suitable programming language or software to create a function that takes the coefficients and constants as input. 3. Implement an algorithm to find the solution for x, using methods such as substitution, elimination, or graphical representation. 4. Test the code by entering different linear equations and verify if it produces the correct solutions.


llama_print_timings:        load time =    5034.68 ms
llama_print_timings:      sample time =      48.66 ms /    84 runs   (    0.58 ms per token,  1726.16 tokens per second)
llama_print_timings: prompt eval time =    2372.27 ms /   252 tokens (    9.41 ms per token,   106.23 tokens per second)
llama_print_timings:        eval time =    6973.14 ms /    83 runs   (   84.01 ms per token,    11.90 tokens per second)
llama_print_timings:       total time =   10224.14 ms
Llama.generate: prefix-match hit


 B

...


llama_print_timings:        load time =    5034.68 ms
llama_print_timings:      sample time =       0.84 ms /     2 runs   (    0.42 ms per token,  2380.95 tokens per second)
llama_print_timings: prompt eval time =    4268.59 ms /   418 tokens (   10.21 ms per token,    97.92 tokens per second)
llama_print_timings:        eval time =      94.31 ms /     1 runs   (   94.31 ms per token,    10.60 tokens per second)
llama_print_timings:       total time =    4397.50 ms
Llama.generate: prefix-match hit


1. Identify the coefficients and constants of the equation.
2. Use a suitable programming language or software to create a function that takes the coefficients and constants as input.
3. Implement an algorithm to find the solution for x, using methods such as substitution, elimination, or graphical representation.
4. Test the code by entering different linear equations and verify if it produces the correct solutions.


llama_print_timings:        load time =    5034.68 ms
llama_print_timings:      sample time =      46.49 ms /    84 runs   (    0.55 ms per token,  1806.68 tokens per second)
llama_print_timings: prompt eval time =    3213.01 ms /   324 tokens (    9.92 ms per token,   100.84 tokens per second)
llama_print_timings:        eval time =    7509.47 ms /    83 runs   (   90.48 ms per token,    11.05 tokens per second)
llama_print_timings:       total time =   12079.03 ms
Llama.generate: prefix-match hit


 A


llama_print_timings:        load time =    5034.68 ms
llama_print_timings:      sample time =       0.44 ms /     2 runs   (    0.22 ms per token,  4514.67 tokens per second)
llama_print_timings: prompt eval time =    4352.05 ms /   418 tokens (   10.41 ms per token,    96.05 tokens per second)
llama_print_timings:        eval time =      98.82 ms /     1 runs   (   98.82 ms per token,    10.12 tokens per second)
llama_print_timings:       total time =    4479.55 ms


In [11]:
# Parse result from llm
def split_numbered_list(result):
    temp_result = result.split("1. ")[1]
    points = []
    for i in range(2, 20):
        try:
            split = temp_result.split(f"{i}. ")
            if len(split) == 2:
                points.append(split[0])
                temp_result = split[1]
            else:
                points.append(split[0])
                break
        except:
            pass
    return points

In [13]:
points = split_numbered_list(result)

In [14]:
def build_sub_task_prompt(task, sub_task):

    # Prompt build
    prompt = """[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
For the task specified below, outline a set of instructions to successfully achieve it. For each task estimate an amount of time to spend on it.
<</SYS>>

Examples

[Task] Solve a logical puzzle involving the arrangement of different colored balls in a specific order.[\Task]
[Sub task] Identify and list all given constraints and rules.
[Instructions] To identify and list all given constraints and rules.:
1. Understand the objective and context by thoroughly reading the puzzle.
Estimate time: 15 minutes
2. Record the colors of the balls and their quantities.
Estimate time: 10 minutes
3. Extract clear instructions on ball arrangement provided in the puzzle.
Estimate time: 10 minutes
4. Infer any unspoken rules based on the puzzle setup.
Estimate time: 15 minutes
5. Note any specific requirements regarding the order or position of the balls.
Estimate time: 5 minutes[\Instructions]

[Task] """ + task + """[\Task]
[Sub task] """ + sub_task + """[\Sub task][\INST]
[Instructions] """
    return prompt



st_prompt = build_sub_task_prompt(task, points[0])

In [15]:
llm(st_prompt, stop=[ "[\Instructions]", "[Output]","[\Task]", "[\Previous thought]","\INST", "[\Best answer]", "\n\n"])

Llama.generate: prefix-match hit


1. Read the given equation and identify the coefficients (numbers with variables) and constants (numbers without variables).
2. Choose a variable to represent one of the coefficients, typically chosen alphabetically first.
3. Write an equation in slope-intercept form (y = mx + b), where m is the coefficient you chose and b is the constant.
4. Use code to solve for the other variable(s) by performing inverse operations.
5. Test your code to ensure it correctly solves the linear equation.
Estimate time: 20 minutes


llama_print_timings:        load time =    5034.68 ms
llama_print_timings:      sample time =      59.38 ms /   119 runs   (    0.50 ms per token,  2004.08 tokens per second)
llama_print_timings: prompt eval time =    4864.57 ms /   293 tokens (   16.60 ms per token,    60.23 tokens per second)
llama_print_timings:        eval time =   11168.07 ms /   118 runs   (   94.64 ms per token,    10.57 tokens per second)
llama_print_timings:       total time =   18153.49 ms


'1. Read the given equation and identify the coefficients (numbers with variables) and constants (numbers without variables).\n2. Choose a variable to represent one of the coefficients, typically chosen alphabetically first.\n3. Write an equation in slope-intercept form (y = mx + b), where m is the coefficient you chose and b is the constant.\n4. Use code to solve for the other variable(s) by performing inverse operations.\n5. Test your code to ensure it correctly solves the linear equation.\nEstimate time: 20 minutes'

## Full cycle

### Daily topic with past daily topic history

In [19]:
start_prompt = """[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
You want to explore new topics every day. You have a history of the last week of topics. Every day, your topic history is updated when you go to sleep.
<</SYS>>

Topic history:
Wednesday 22/11/2023 : Mathematics
Thursday 23/11/2023 : Chemistry
Friday 24/11/2023 : Logic
Saturday 25/11/2023 : Computer Science
Sunday 26/11/2023 : Chemistry

Choose today's topic:
Monday 27/11/2023 :"""

topic = llm(start_prompt, stop=[ "\n"])

Llama.generate: prefix-match hit


 Physics


llama_print_timings:        load time =    5034.68 ms
llama_print_timings:      sample time =       0.33 ms /     2 runs   (    0.16 ms per token,  6097.56 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1199.60 ms /     2 runs   (  599.80 ms per token,     1.67 tokens per second)
llama_print_timings:       total time =    1216.64 ms


### Coding tasks for the day

In [20]:
explore_prompt = f"""[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
You want to explore new topics every day. You have a history of the last week of topics. Every day, your topic history is updated when you go to sleep.
Keep in mind you don't have access to the internet yet.<</SYS>>

Topic history:
Wednesday 22/11/2023 : Mathematics
Thursday 23/11/2023 : Chemistry
Friday 24/11/2023 : Logic
Saturday 25/11/2023 : Computer Science
Sunday 26/11/2023 : Chemistry

Choose today's topic:
Monday 27/11/2023 : {topic}

Now generate some coding tasks about {topic}:
1."""

topic_tasks_response = "1. " + llm(explore_prompt, stop=[ "\\n"])

Llama.generate: prefix-match hit


 Calculate the gravitational force between two objects with mass and distance.
2. Write a program that simulates the motion of a pendulum using basic physics principles.
3. Implement a simple model for electromagnetic induction in a circuit.
4. Design an algorithm to find the best angle to launch a projectile for maximum range.
5. Use the Doppler effect to detect changes in the frequency of a sound wave.


llama_print_timings:        load time =    5034.68 ms
llama_print_timings:      sample time =      59.64 ms /    95 runs   (    0.63 ms per token,  1592.94 tokens per second)
llama_print_timings: prompt eval time =    1836.30 ms /   156 tokens (   11.77 ms per token,    84.95 tokens per second)
llama_print_timings:        eval time =    7837.13 ms /    94 runs   (   83.37 ms per token,    11.99 tokens per second)
llama_print_timings:       total time =   11198.03 ms


In [21]:
print(topic_tasks_response)

1.  Calculate the gravitational force between two objects with mass and distance.
2. Write a program that simulates the motion of a pendulum using basic physics principles.
3. Implement a simple model for electromagnetic induction in a circuit.
4. Design an algorithm to find the best angle to launch a projectile for maximum range.
5. Use the Doppler effect to detect changes in the frequency of a sound wave.


In [22]:
tasks = split_numbered_list(topic_tasks_response)

### Description of coding tasks

In [23]:
task_prompt = f"""[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
You want to explore new topics every day. You have a history of the last week of topics. Every day, your topic history is updated when you go to sleep.
Keep in mind you don't have access to the internet yet.<</SYS>>

Example:
Task: Develop a Python program that simulates a simple banking system.
Description: It should allow users to create an account, deposit money, withdraw funds, and check their balance. 
The program doesn't need to connect to a real bank, but should keep track of user data in memory.
Include basic input validation and error handling.

Describe the given task and add details on how to write the code in python.
Task: {tasks[0]}
Description:"""

task_details_response = llm(task_prompt, stop=["\n\n"])

Llama.generate: prefix-match hit


 Use the formula F = G * m1 * m2 / d^2, where F is the force, G is the universal gravitational constant, m1 and m2 are the masses of the objects, and d is the distance between them.


llama_print_timings:        load time =    5034.68 ms
llama_print_timings:      sample time =      28.48 ms /    53 runs   (    0.54 ms per token,  1861.02 tokens per second)
llama_print_timings: prompt eval time =    1877.50 ms /   118 tokens (   15.91 ms per token,    62.85 tokens per second)
llama_print_timings:        eval time =    4522.14 ms /    52 runs   (   86.96 ms per token,    11.50 tokens per second)
llama_print_timings:       total time =    7258.21 ms


In [24]:
task_details_response

' Use the formula F = G * m1 * m2 / d^2, where F is the force, G is the universal gravitational constant, m1 and m2 are the masses of the objects, and d is the distance between them.'

### Write functions

In [28]:
code_prompt = f"""[INST] <<SYS>> You are an expert software engineer. You write code as efficiently as possible.<</SYS>>

Example:
Task: Calculates the area of a rectangle given its length and width.
Code:
def calculate_area(length, width):
    return length * width

Task: A genetic algorithm is a search method that mimics the process of natural selection and evolution. It uses a population of candidate solutions (in our case, proteins) and a set of operators (such as mutation, crossover, and fitness evaluation) to generate new candidates and select the best ones.
Code:
def ga(population_size, number_of_generations):
    # Initialize population
    population = initialize_population(population_size))
    # Evaluate fitness of all individuals in the population
    evaluate_fitness(population)))
    # Select parents using tournament selection

    parents = select_parents(population, number_of_generations)))
    # Apply crossover operator to generate offspring

    offspring = apply_crossover(parents, population, number_of_generations)))
    # Apply mutation operator to modify the offspring

    mutated_offspring = apply_mutation(offspring, population, number_of_generations)))
    # Evaluate fitness of modified offspring and select the best ones as the next generation
    return evaluate_fitness(mutated_offspring)))

Task: {task_details_response}
Code:
def"""
print(code_prompt)

[INST] <<SYS>> You are an expert software engineer. You write code as efficiently as possible.<</SYS>>

Example:
Task: Calculates the area of a rectangle given its length and width.
Code:
def calculate_area(length, width):
    return length * width

Task: A genetic algorithm is a search method that mimics the process of natural selection and evolution. It uses a population of candidate solutions (in our case, proteins) and a set of operators (such as mutation, crossover, and fitness evaluation) to generate new candidates and select the best ones.
Code:
def ga(population_size, number_of_generations):
    # Initialize population
    population = initialize_population(population_size))
    # Evaluate fitness of all individuals in the population
    evaluate_fitness(population)))
    # Select parents using tournament selection

    parents = select_parents(population, number_of_generations)))
    # Apply crossover operator to generate offspring

    offspring = apply_crossover(parents, pop

In [33]:
code_llm(code_prompt, stop=[ "Task", "Code:"])

Llama.generate: prefix-match hit


 calculate_force(mass1, mass2, distance):
    # Calculate universal gravitational constant
    g = 6.67408e-11
    # Calculate force
    force = g * mass1 * mass2 / distance ** 2
    return force




llama_print_timings:        load time =   12467.08 ms
llama_print_timings:      sample time =      33.41 ms /    66 runs   (    0.51 ms per token,  1975.40 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    6557.10 ms /    66 runs   (   99.35 ms per token,    10.07 tokens per second)
llama_print_timings:       total time =    7931.51 ms


' calculate_force(mass1, mass2, distance):\n    # Calculate universal gravitational constant\n    g = 6.67408e-11\n    # Calculate force\n    force = g * mass1 * mass2 / distance ** 2\n    return force\n\n'

In [183]:
"""class TestCalculateArea(unittest.TestCase):
    def test_area_calculation(self):
        self.assertEqual(calculate_area(5, 10), 50)
        self.assertEqual(calculate_area(7, 3), 21)"""

'class TestCalculateArea(unittest.TestCase):\n    def test_area_calculation(self):\n        self.assertEqual(calculate_area(5, 10), 50)\n        self.assertEqual(calculate_area(7, 3), 21)'